### Import dependencies

In [1]:

import openai
import instructor
from qdrant_client import QdrantClient

from pydantic import BaseModel, Field

### mocking the prompt

In [2]:
prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: What is your name?
"""

In [3]:
openai_response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0
)

print(openai_response.choices[0].message.content)

I am ChatGPT, your AI assistant. How can I help you today?


### Add Instructor

In [4]:
client = instructor.from_openai(openai.OpenAI())

In [5]:
class RAGGenerationResponse(BaseModel):
    answer: str =Field(description="the answer to the question")

In [7]:
response, raw_response= client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role":"system", "content": prompt}
    ],
    temperature=0,
    response_model= RAGGenerationResponse
)

In [8]:
response

RAGGenerationResponse(answer='I am ChatGPT, an AI language model created by OpenAI. How can I assist you today?')

In [9]:
raw_response

ChatCompletion(id='chatcmpl-D1wKO8DKf9Sles1ulmEy88h9PWDre', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_GP1Qg7OyaziUXRHRGNjNnNik', function=Function(arguments='{"answer":"I am ChatGPT, an AI language model created by OpenAI. How can I assist you today?"}', name='RAGGenerationResponse'), type='function')]))], created=1769354912, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=25, prompt_tokens=95, total_tokens=120, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [10]:
class RAGGenerationResponse(BaseModel):
    answer: str =Field(description="the answer to the question")
    reasoning: str=Field(description="The reasoning for the answer")

In [11]:
response, raw_response= client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role":"system", "content": prompt}
    ],
    temperature=0,
    response_model= RAGGenerationResponse
)

In [12]:
response

RAGGenerationResponse(answer='My name is ChatGPT.', reasoning='I am an AI language model created by OpenAI, and I am commonly known as ChatGPT.')

### RAG Example

In [13]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [14]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""
    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):
    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "datamodel": answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [15]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [16]:
output = rag_pipeline("Can I get a charging cable? Please suggest me a good one.", qdrant_client)

In [17]:
output

{'datamodel': RAGGenerationResponse(answer='Yes, you can get a charging cable with the Siniffo Upgraded Bone Conduction Headphones (ID: B0BNHVLF7G). This product includes a Type-C quick charging cable along with the headphones. It is a good choice if you are looking for a high-quality sport headphone with a charging cable included.'),
 'answer': 'Yes, you can get a charging cable with the Siniffo Upgraded Bone Conduction Headphones (ID: B0BNHVLF7G). This product includes a Type-C quick charging cable along with the headphones. It is a good choice if you are looking for a high-quality sport headphone with a charging cable included.',
 'question': 'Can I get a charging cable? Please suggest me a good one.',
 'retrieved_context_ids': ['B09PRK49JH',
  'B0BNHVLF7G',
  'B0CGM29ZL7',
  'B08BR9X387',
  'B0BB1YKXL1'],
 'retrieved_context': ['YUNZII AKKO Neon 157 Keycaps ASA Profile Keycap Set Double-Shot PBT Keycap Gift Set for Mechanical Keyboard (157 Keys, ASA NEON) 【Stand Out Aesthetic Color

In [18]:
print(output["answer"])

Yes, you can get a charging cable with the Siniffo Upgraded Bone Conduction Headphones (ID: B0BNHVLF7G). This product includes a Type-C quick charging cable along with the headphones. It is a good choice if you are looking for a high-quality sport headphone with a charging cable included.
